In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.3'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [77.5 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,659 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,906 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,715 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:13 http://archive.ubuntu.com

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header = True, inferSchema = True, quote = "\"", escape = "\"")

df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("homes")

In [8]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("SELECT ROUND(AVG(price), 2) AS Average_Price FROM homes WHERE bedrooms = 4").show()

+-------------+
|Average_Price|
+-------------+
|    299661.01|
+-------------+



In [7]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("SELECT ROUND(AVG(price), 2) AS Average_Price, YEAR(date) AS Year FROM homes WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY YEAR(date) ORDER BY YEAR(date) DESC").show()

+-------------+----+
|Average_Price|Year|
+-------------+----+
|    292725.69|2022|
|    294211.46|2021|
|    294204.16|2020|
|    287287.82|2019|
+-------------+----+



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("SELECT ROUND(AVG(price), 2) AS Average_Price, YEAR(date) AS Year FROM homes WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_lot >= 2000 GROUP BY YEAR(date) ORDER BY YEAR(date) DESC").show()

+-------------+----+
|Average_Price|Year|
+-------------+----+
|    287162.18|2022|
|    290319.06|2021|
|    293176.41|2020|
|    282752.72|2019|
+-------------+----+



In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT ROUND(AVG(price), 2) AS Average_Price, view FROM homes WHERE price >= 350000 GROUP BY view ORDER BY view DESC").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_Price|view|
+-------------+----+
|    1026669.5| 100|
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
+-------------+----+
only showing top 20 rows

--- 0.7147283554077148 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE homes")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('homes')

True

In [13]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("SELECT ROUND(AVG(price), 2) AS Average_Price, view FROM homes WHERE price >= 350000 GROUP BY view ORDER BY view DESC").show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|Average_Price|view|
+-------------+----+
|    1026669.5| 100|
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
+-------------+----+
only showing top 20 rows

--- 0.5168383121490479 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet("parquet_homes", mode = "overwrite")

In [15]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet("parquet_homes")

In [16]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView("p_homes")

In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("SELECT ROUND(AVG(price), 2) AS Average_Price, view FROM p_homes WHERE price >= 350000 GROUP BY view ORDER BY view DESC").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_Price|view|
+-------------+----+
|    1026669.5| 100|
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
+-------------+----+
only showing top 20 rows

--- 0.4170248508453369 seconds ---


In [18]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE homes")

DataFrame[]

In [19]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('homes')

False